twitter, elon musk buyout, delist, name change to X.com

In [9]:
import pandas as pd
import numpy as np
import yfinance as yf
import pandas_datareader as pdr
from datetime import datetime, timedelta
import requests
import warnings
warnings.filterwarnings('ignore')

In [11]:
def get_last_delisted_data(ticker, delisting_date):
    """
    Fetch last available data for a delisted stock from Yahoo Finance.
    
    Parameters:
    -----------
    ticker : str
        Stock ticker symbol
    delisting_date : str
        Date of delisting in 'YYYY-MM-DD' format
    
    Returns:
    --------
    dict
        Contains last available price data and company information
    """
    try:
        # Initialize ticker object
        stock = yf.Ticker(ticker)
        
        # Get end date and lookback period
        end_date = pd.to_datetime(delisting_date)
        start_date = end_date - pd.Timedelta(days=5)  # Look back 5 days to ensure we get last data
        
        # Get historical data
        hist = stock.history(
            start=start_date.strftime('%Y-%m-%d'),
            end=end_date.strftime('%Y-%m-%d')
        )
        
        # Get last available data point
        if not hist.empty:
            last_data = hist.iloc[-1]
            
            # Get company info
            info = stock.info
            
            result = {
                'ticker': ticker,
                'delisting_date': delisting_date,
                'last_trading_data': {
                    'date': hist.index[-1].strftime('%Y-%m-%d'),
                    'close': last_data['Close'],
                    'volume': last_data['Volume'],
                    'high': last_data['High'],
                    'low': last_data['Low'],
                    'open': last_data['Open']
                },
                'company_info': {
                    'name': info.get('longName', 'N/A'),
                    'industry': info.get('industry', 'N/A'),
                    'sector': info.get('sector', 'N/A'),
                    'employees': info.get('fullTimeEmployees', 'N/A'),
                    'officers': info.get('companyOfficers', [])
                }
            }
            
            return result
        else:
            print(f"No historical data found for {ticker}")
            return None
            
    except Exception as e:
        print(f"Error fetching data for {ticker}: {str(e)}")
        return None


$TWTR: possibly delisted; no timezone found


No historical data found for TWTR
No data available


In [ ]:
# Example usage
ticker = "TWTR"
delisting_date = "2022-10-27"

data = get_last_delisted_data(ticker, delisting_date)

if data:
    print(f"\n=== Last Available Data for {data['ticker']} ===")
    print(f"Company: {data['company_info']['name']}")
    print(f"Delisting Date: {data['delisting_date']}")
    
    print("\nLast Trading Data:")
    for key, value in data['last_trading_data'].items():
        if isinstance(value, float):
            print(f"{key.title()}: ${value:,.2f}" if key != 'volume' else f"{key.title()}: {value:,.0f}")
        else:
            print(f"{key.title()}: {value}")
    
    print("\nCompany Officers:")
    for officer in data['company_info']['officers']:
        print(f"Name: {officer.get('name', 'N/A')}")
        print(f"Title: {officer.get('title', 'N/A')}")
        if 'totalPay' in officer:
            print(f"Total Pay: ${officer['totalPay']:,.2f}")
        print("---")
else:
    print("No data available")